# Проект для «Викишоп»

Интернет-магазин «Викишоп» запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Обучите модель классифицировать комментарии на позитивные и негативные. В вашем распоряжении набор данных с разметкой о токсичности правок.

Постройте модель со значением метрики качества *F1* не меньше 0.75. 

**Инструкция по выполнению проекта**

1. Загрузите и подготовьте данные.
2. Обучите разные модели. 
3. Сделайте выводы.

Для выполнения проекта применять *BERT* необязательно, но вы можете попробовать.

**Описание данных**

Данные находятся в файле `toxic_comments.csv`. Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

## Импортирование библиотек 

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from nltk.stem import WordNetLemmatizer 
import nltk
from nltk.corpus import stopwords
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier
from sklearn.feature_extraction.text import TfidfVectorizer

## Подготовка

In [2]:
#загружаю датасет
try:
    data = pd.read_csv('/Users/kostyatrufanov/Downloads/toxic_comments.csv')
except:
    data = pd.read_csv('/datasets/toxic_comments.csv')
    
data.head()

,Unnamed: 0,text,toxic
0,0,Explanation\nWhy the edits made under my usern...,0
1,1,D'aww! He matches this background colour I'm s...,0
2,2,"Hey man, I'm really not trying to edit war. It...",0
3,3,"""\nMore\nI can't make any real suggestions on ...",0
4,4,"You, sir, are my hero. Any chance you remember...",0


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159292 entries, 0 to 159291
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   Unnamed: 0  159292 non-null  int64 
 1   text        159292 non-null  object
 2   toxic       159292 non-null  int64 
dtypes: int64(2), object(1)
memory usage: 3.6+ MB


In [4]:
#поменяю название первого признака
data.columns = ['number', 'text', 'toxic']

In [5]:
#разделю таблицу 
data_train, data_temporary = train_test_split(data, test_size=0.4)
data_valid, data_test = train_test_split(data_temporary, test_size=0.5)
#проверяю
display(data_train.shape)
display(data_valid.shape)
data_test.shape

(95575, 3)

(31858, 3)

(31859, 3)

In [6]:
#создам функцию для очистки и лемматизации 
from nltk.corpus import wordnet
nltk.download('averaged_perceptron_tagger')

def get_wordnet_pos(word):
    tag = nltk.pos_tag([word])[0][1][0].upper()
    tag_dict = {"J": wordnet.ADJ,
                "N": wordnet.NOUN,
                "V": wordnet.VERB,
                "R": wordnet.ADV}
    return tag_dict.get(tag, wordnet.NOUN)

def lemmatize(corpus):
    m = WordNetLemmatizer()
    lemm_text = ' '.join(m.lemmatize(w, get_wordnet_pos(w)) for w in nltk.word_tokenize(corpus))
    return lemm_text

def clear_text(corpus):
    lemm_clear = re.sub(r'[^a-zA-Z]', ' ', corpus)
    lemm_clear = lemm_clear.split()
    lemm_clear = ' '.join(lemm_clear)
    return lemm_clear

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/kostyatrufanov/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [13]:
import nltk
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
nltk.download('wordnet')

[nltk_data] Downloading package punkt_tab to
[nltk_data]     /Users/kostyatrufanov/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /Users/kostyatrufanov/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kostyatrufanov/nltk_data...


True

In [7]:
#очищаю обучащую выборку
data_train['text'] = data_train['text'].apply(clear_text)

In [14]:
#лемматизирую обучающую выборку 
data_train['lemm_text'] = data_train['text'].apply(lemmatize)

In [15]:
data_train.head()

,number,text,toxic,lemm_text
112234,112331,October Please stop your disruptive editing If...,0,October Please stop your disruptive edit If yo...
53212,53273,I think this addition adequately explains oppo...,0,I think this addition adequately explains oppo...
69405,69473,I ve corrected the climate records for the cha...,0,I ve correct the climate record for the chart ...
145093,145249,Low quality and uninformative This article is ...,0,Low quality and uninformative This article be ...
56205,56266,Series episode,0,Series episode


In [16]:
#очищаю валидационную выборку
data_valid['text'] = data_valid['text'].apply(clear_text)
#лемматизирую валидационную выборку 
data_valid['lemm_text'] = data_valid['text'].apply(lemmatize)

In [17]:
#очищаю тестовую выборку
data_test['text'] = data_test['text'].apply(clear_text)
#лемматизирую тестовую выборку 
data_test['lemm_text'] = data_test['text'].apply(lemmatize)

In [21]:
#загрузжаю список стоп-слов в английском языке 
nltk.download('stopwords')
stop_words = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/kostyatrufanov/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
#создаю счетчик
tfidf = TfidfVectorizer(stop_words=stop_words)

In [23]:
#превращаю слова в векторы, чтобы можно было обучить и предсказать
features_train = tfidf.fit_transform(data_train['lemm_text'])
features_valid = tfidf.transform(data_valid['lemm_text'])
features_test = tfidf.transform(data_test['lemm_text'])

In [24]:
#отделяю целевой признак от датафреймов
target_train = data_train['toxic']
target_valid = data_valid['toxic']
target_test = data_test['toxic']

In [25]:
#проверяю
display(features_train.shape)
display(target_train.shape)

display(features_valid.shape)
display(target_valid.shape)

display(features_test.shape)
target_test.shape

(95575, 117736)

(95575,)

(31858, 117736)

(31858,)

(31859, 117736)

(31859,)

### Вывод: Загрузил датафрейм, поменял названия признаков. Затем разделил основную выборку на три, потом каждую преобразовал в векторы, убрав лишние слова 

## Обучение

**Начну с Логистической регрессии**

In [26]:
grid_space = {'random_state':[12345],
             'max_iter':[200],
             'penalty' : ['l2'],
             'C':[1, 4, 7, 10, 13, 15]}
#создаю модель
model = LogisticRegression()
grid = GridSearchCV(model, param_grid=grid_space, cv=3, error_score='raise')
model_grid = grid.fit(features_train, target_train)
#сохраняю наилучшии параметры
best_parametrs = model_grid.best_params_
best_parametrs

{'C': 13, 'max_iter': 200, 'penalty': 'l2', 'random_state': 12345}

In [27]:
#считаю f1 на валид. выборке
model = LogisticRegression(random_state=12345, C = 10, penalty = 'l2', solver='liblinear', max_iter=200)
model.fit(features_train, target_train)
predict = model.predict(features_valid)
f1_score(target_valid, predict)

0.7667069590744259

**Теперь catboost**

In [28]:
grid_space = {'iterations':[100, 200, 300], 
    'learning_rate':[0.1, 0.2, 0.3, 0.4, 0.5],
    'verbose':[50]}
#создаю модель
model = CatBoostClassifier()
grid = GridSearchCV(model, param_grid=grid_space, cv=3)
model_grid = grid.fit(features_train, target_train)
#сохраняю наилучшии параметры
best_parametrs = model_grid.best_params_
best_parametrs

0:	learn: 0.5946782	total: 217ms	remaining: 21.4s
50:	learn: 0.1911836	total: 5.64s	remaining: 5.42s
99:	learn: 0.1683203	total: 11s	remaining: 0us
0:	learn: 0.5974153	total: 119ms	remaining: 11.8s
50:	learn: 0.1899543	total: 5.42s	remaining: 5.2s
99:	learn: 0.1675183	total: 10.5s	remaining: 0us
0:	learn: 0.5944308	total: 134ms	remaining: 13.3s
50:	learn: 0.1916346	total: 5.42s	remaining: 5.21s
99:	learn: 0.1690202	total: 10.5s	remaining: 0us
0:	learn: 0.5118422	total: 119ms	remaining: 11.7s
50:	learn: 0.1660226	total: 5.41s	remaining: 5.2s
99:	learn: 0.1428275	total: 10.5s	remaining: 0us
0:	learn: 0.5166923	total: 123ms	remaining: 12.2s
50:	learn: 0.1656911	total: 5.42s	remaining: 5.21s
99:	learn: 0.1422904	total: 10.6s	remaining: 0us
0:	learn: 0.5115204	total: 122ms	remaining: 12.1s
50:	learn: 0.1663767	total: 5.39s	remaining: 5.18s
99:	learn: 0.1432882	total: 10.6s	remaining: 0us
0:	learn: 0.4437425	total: 119ms	remaining: 11.8s
50:	learn: 0.1511326	total: 5.33s	remaining: 5.12s
99:

{'iterations': 300, 'learning_rate': 0.4, 'verbose': 50}

In [29]:
#считаю f1 на валид. выборке
model = CatBoostClassifier(iterations=300, learning_rate=0.4, verbose=50)
model.fit(features_train, target_train)
predict = model.predict(features_valid)
f1_score(target_valid, predict)

0:	learn: 0.3868009	total: 204ms	remaining: 1m 1s
50:	learn: 0.1425861	total: 7.13s	remaining: 34.8s
100:	learn: 0.1230672	total: 13.9s	remaining: 27.4s
150:	learn: 0.1128865	total: 20.8s	remaining: 20.5s
200:	learn: 0.1034738	total: 27.6s	remaining: 13.6s
250:	learn: 0.0964152	total: 34.4s	remaining: 6.71s
299:	learn: 0.0918096	total: 41.1s	remaining: 0us


0.7548761201897733

### Вывод: Логистическая регрессия справилась с задачей лучше чем catboost, так что предсказывать на тестовой выборке будет Логистическая регрессия

## Выводы

In [30]:
#считаю f1 на тестовой выборке
model = LogisticRegression(random_state=12345, C = 10, penalty = 'l2', solver='liblinear', max_iter=200)
model.fit(features_train, target_train)
predict = model.predict(features_test)
f1_score(target_test, predict)

0.7638764947349634

## Вывод: Загрузил данные, поменял названиия признаков, затем сделал три корпуса и каждый преобразовал в векторы. После этого проверил две модели, лучше оказалась Логистическая регрессия, и на тестовой выборке она показала результат в 0.77, что удовлетворяет запросу клиента 